In [1]:
import regex as re
import pandas as pd
import os
import random
import numpy as np
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='your module name')

#### 加载自制中文数据集、分词、去除停用词

In [2]:
data = pd.read_csv('smalldata_washed.csv')
rumor = data['rumor'].to_list()
reverse = data['reverse'].to_list()
rumor_class = len(rumor)*[1]
reverse_class = len(reverse)*[0]
data = rumor + reverse
data_class = rumor_class + reverse_class
#后续添加数据要放在raw目录下，使用.xlsx格式将后续标注的数据加入data
raw_data_list = os.listdir('raw')
data_list = []
good_name = re.compile(r'^(?!(\~\$)).*(.xlsx)')
for i in raw_data_list:
    if good_name.match(i):
    #    data_list.append(i)
        temp = pd.read_excel('raw/'+i)
        temp.fillna('',inplace=True)
        temp_rumor = [x.strip() for x in  temp['punc_rumor'].to_list()  if x.strip()!='']
        temp_reverse = [x.strip() for x in  temp['punc_truth'].to_list()  if x.strip()!='']
        temp_rumor_class = len(temp_rumor)*[1]
        temp_reverse_class = len(temp_reverse)*[0]
        temp_data = temp_rumor + temp_reverse
        temp_data_class = temp_rumor_class + temp_reverse_class
        for i in range( len(temp_data)):
            temp_data[i]=re.sub(r'[\,\'\ ]|(\n)','',temp_data[i])
    data = data + temp_data
    data_class = data_class + temp_data_class
with open('data.txt','w') as f:
    for i in range(len(data)):
        f.write(data[i]+'\t'+str(data_class[i])+'\n')
        data_list =list( zip(data, data_class))
random.shuffle(data_list)
data, data_class = zip(*data_list)
index = ['rumor{}{}'.format(i, '!' * j) for (i, j) in zip(range(len(data)), data_class)]
df = pd .DataFrame(data=zip(data_class,data), columns=["rumor","text"], index=index)

# 去除停用词
with open ('hit_stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.read().splitlines()

# jieba分词
import jieba
for i in range(len(df)):
    words = jieba.cut(df["text"].iloc[i],cut_all=False)
    words = [word for word in words if word not in stopwords]
    df["text"].iloc[i] = ' '.join(words) 

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/f0/6j6j3w814n53skbt2s4d8p5m0000gn/T/jieba.cache
Loading model cost 0.503 seconds.
Prefix dict has been built successfully.
/var/folders/f0/6j6j3w814n53skbt2s4d8p5m0000gn/T/ipykernel_21518/2493417845.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"].iloc[i] = ' '.join(words)


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import casual_tokenize
tfidf = TfidfVectorizer(tokenizer=casual_tokenize)
tfidf_docs = tfidf.fit_transform(raw_documents=df.text).toarray()
len(tfidf.vocabulary_)

/Users/stevenyu/opt/anaconda3/envs/d2l/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


9200

In [4]:
tfidf_docs = pd.DataFrame(tfidf_docs)
#通过减去每个文档（词袋向量）的平均值来中心化文档
tfidf_docs = tfidf_docs - tfidf_docs.mean()
tfidf_docs.shape
df.rumor.sum()

913

#### 基于PCA的短消息语义分析

In [5]:
from sklearn.decomposition import PCA
k=min(len(tfidf.vocabulary_),len(df))
k=300
pca = PCA(n_components=k)
pca_docs = pca.fit_transform(tfidf_docs)
base = sum(pca.singular_values_)


In [6]:
nums = pca.singular_values_.tolist()

In [7]:
'''
for i in range(len(nums)):
    if(sum(nums[:i+1])/base>0.5):
        k=i
        break
'''


'\nfor i in range(len(nums)):\n    if(sum(nums[:i+1])/base>0.5):\n        k=i\n        break\n'

In [8]:
pca = PCA(n_components=k)
pca_docs = pca.fit_transform(tfidf_docs)
pca_topic_vectors = pca.transform(tfidf_docs)
columns = ['topic{}'.format(i) for i in range(pca_docs.shape[1])]
pca_topic_vectors = pd.DataFrame(pca_topic_vectors, columns=columns, index=index)
pca_topic_vectors.round(3).head(6)

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,...,topic290,topic291,topic292,topic293,topic294,topic295,topic296,topic297,topic298,topic299
rumor0!,0.062,-0.020,0.056,-0.095,-0.036,0.083,0.044,0.023,-0.042,0.009,...,0.010,0.010,-0.005,-0.024,0.033,0.069,0.016,0.002,0.032,0.019
rumor1!,0.029,-0.047,-0.022,0.017,-0.026,-0.063,-0.011,-0.009,-0.017,0.001,...,-0.025,0.053,0.046,0.021,0.016,-0.024,0.009,-0.018,-0.016,-0.024
rumor2,0.055,-0.014,0.019,-0.000,0.018,0.075,-0.050,0.020,0.008,-0.029,...,-0.010,-0.018,0.019,-0.013,0.008,0.008,0.054,0.014,-0.007,0.025
rumor3!,0.003,0.073,-0.045,0.078,0.051,-0.004,-0.051,-0.105,-0.008,0.104,...,0.032,-0.020,-0.017,-0.011,-0.018,0.004,0.014,-0.016,-0.006,0.033
rumor4,0.027,-0.029,-0.026,0.018,-0.032,-0.052,0.001,0.001,-0.031,-0.004,...,0.027,-0.000,0.002,-0.016,0.006,0.009,-0.021,0.021,-0.054,0.025
rumor5!,0.151,0.020,0.060,0.016,-0.050,0.020,0.028,0.008,-0.019,0.010,...,0.021,-0.039,-0.009,-0.008,-0.011,-0.028,0.005,-0.015,-0.000,-0.026


In [9]:
tfidf.vocabulary_
#根据词项的频率对词汇表进行排序
#当对某个不按照最左边元素排序的序列解压并在排序后重新压缩时，可以使用zip(*sorted(zip(...)))
column_nums , terms = zip(*sorted(zip(tfidf.vocabulary_.values(), tfidf.vocabulary_.keys())))


In [10]:
weights = pd.DataFrame(pca.components_, columns=terms,
                       index = ['topic{}'.format(i) for i in range(pca.components_.shape[0])])
pd.options.display.max_columns = 8
weights.head(4).round(3)

,%,.,0,0t,...,鼻炎,鼻翼,龋齿,龙葵
topic0,0.001,-0.001,0.0,-0.000,...,-0.000,0.000,0.001,-0.001
topic1,-0.001,0.000,-0.0,-0.001,...,-0.002,-0.000,-0.001,-0.002
topic2,-0.001,0.001,-0.0,-0.001,...,-0.002,-0.001,-0.001,0.000
topic3,0.000,0.004,0.0,0.001,...,-0.002,0.001,0.001,-0.002


#### 基于截断的SVD的短消息语义分析（recommend）

In [21]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=k,n_iter=100)
svd_topic_vectors = svd.fit_transform(tfidf_docs.values)
svd_topic_vectors = pd.DataFrame(svd_topic_vectors, columns=columns, index=index)
svd_topic_vectors.round(3).head(6)

In [ ]:

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(svd_topic_vectors,df.rumor,test_size=0.2 ,random_state=42)
smote = SMOTE(random_state=42)
# 进行过采样
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print("过采样前训练集中类别0的数量：", sum(y_train==0))
print("过采样前训练集中类别1的数量：", sum(y_train==1))

print("过采样后训练集中类别0的数量：", sum(y_train_res==0))
print("过采样后训练集中类别1的数量：", sum(y_train_res==1))
lda = LDA(n_components=1)
lda = lda.fit(X_train_res, y_train_res)
df['LSA16_rumor'] = lda.predict(svd_topic_vectors)
round(float(lda.score(X_test, y_test)), 3)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(svd_topic_vectors,df.rumor,test_size=0.2 ,random_state=42)
smote = SMOTE(random_state=42)
# 进行过采样
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print("过采样前训练集中类别0的数量：", sum(y_train==0))
print("过采样前训练集中类别1的数量：", sum(y_train==1))

print("过采样后训练集中类别0的数量：", sum(y_train_res==0))
print("过采样后训练集中类别1的数量：", sum(y_train_res==1))
lda = LDA(n_components=1)
lda = lda.fit(X_train_res, y_train_res)
df['LSA16_rumor'] = lda.predict(svd_topic_vectors)
round(float(lda.score(X_test, y_test)), 3)

过采样前训练集中类别0的数量： 1559
过采样前训练集中类别1的数量： 433
过采样后训练集中类别0的数量： 1559
过采样后训练集中类别1的数量： 1559


0.801

In [22]:
import numpy as np
svd_topic_vectors = (svd_topic_vectors.T / np.linalg.norm(svd_topic_vectors,axis=1)).T
svd_topic_vectors.iloc[:10].dot(svd_topic_vectors.iloc[:10].T).round(1)

,rumor0!,rumor1,rumor2!,rumor3,...,rumor6,rumor7!,rumor8!,rumor9
rumor0!,1.0,-0.1,0.0,-0.0,...,-0.0,-0.0,0.0,0.0
rumor1,-0.1,1.0,-0.1,-0.1,...,0.1,-0.0,-0.1,-0.1
rumor2!,0.0,-0.1,1.0,0.0,...,0.0,-0.0,0.0,-0.1
rumor3,-0.0,-0.1,0.0,1.0,...,-0.1,0.4,-0.0,0.0
rumor4,-0.0,-0.1,0.3,-0.1,...,-0.1,0.1,0.0,-0.1
rumor5,-0.0,0.3,0.1,-0.1,...,-0.1,-0.0,-0.0,-0.0
rumor6,-0.0,0.1,0.0,-0.1,...,1.0,0.2,-0.0,-0.0
rumor7!,-0.0,-0.0,-0.0,0.4,...,0.2,1.0,0.0,0.0
rumor8!,0.0,-0.1,0.0,-0.0,...,-0.0,0.0,1.0,-0.0
rumor9,0.0,-0.1,-0.1,0.0,...,-0.0,0.0,-0.0,1.0


In [23]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(lda, svd_topic_vectors, df.rumor, cv=5)
print("rumor Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))


NameError: name 'lda' is not defined

In [ ]:
from sklearn.metrics import f1_score, precision_recall_fscore_support
y_true = df.rumor
y_pred = df.LSA16_rumor
p_macro, r_macro, f_macro, support_macro \
    = precision_recall_fscore_support(y_true=y_true, y_pred=y_pred, labels=[0,1], average='macro')

p_micro, r_micro, f_micro, support_micro\
    = precision_recall_fscore_support(y_true=y_true, y_pred=y_pred, labels=[0,1], average='micro')

print('macro: p {}, r {}, f1 {}'.format(p_macro, r_macro, f_macro))

print('micro: p {}, r {}, f1 {}'.format(p_micro, r_micro, f_micro))

macro: p 0.7554878323378912, r 0.8471424815354829, f1 0.7749943201355496
micro: p 0.8164658634538152, r 0.8164658634538152, f1 0.8164658634538152


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import metrics
import time
from tensorboardX import SummaryWriter

y_true = df.loc[y_test.index].rumor
y_pred = df.loc[y_test.index].LSA16_rumor
loss_total = 0
acc = metrics.accuracy_score(y_true, y_pred)
report = metrics.classification_report(y_true, y_pred, target_names=["non-rumor","rumor"], digits=4)
confusion = metrics.confusion_matrix(y_true, y_pred)
print(report)

/Users/stevenyu/opt/anaconda3/envs/d2l/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


              precision    recall  f1-score   support

   non-rumor     0.9569    0.7854    0.8627       396
       rumor     0.5087    0.8627    0.6400       102

    accuracy                         0.8012       498
   macro avg     0.7328    0.8240    0.7513       498
weighted avg     0.8651    0.8012    0.8171       498

